In [2]:
# Chia bộ dữ liệu
from xgboost import XGBRegressor
from mealpy import FloatVar, WOA
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv('Data1.csv')

# Chia thành input và output:
input_data = data.iloc[:, :-1]
output_data = data.iloc[:, -1]

# Chia thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.15, random_state=37)

# Normalize dữ liệu:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
joblib.dump(scaler, 'scaler')
X_test = scaler.transform(X_test)
# Hàm mục tiêu
def objective_func(x):
    n_estimators = int(x[0])
    learning_rate = x[1]
    max_depth = int(x[2])
    # min_child_weight = x[3]
    gamma = x[3]

    model = XGBRegressor(
        n_estimators=n_estimators, learning_rate=learning_rate,
        max_depth=max_depth, #min_child_weight=min_child_weight,
        gamma=gamma, random_state=42
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores_rmse = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
    return round(cv_scores_rmse.mean(), 2)

# Giới hạn các siêu tham số
lb = [50, 0.01, 1, 0]  # n_estimators, learning_rate, max_depth, min_child_weight, gamma
ub = [200, 0.3, 10, 1]

problem_dict = {
    "obj_func": objective_func,
    "bounds": FloatVar(lb=lb, ub=ub),
    "minmax": "max",
    "save_population": True
}

term_dict = {"max_early_stop": 10}

optimizer = WOA.OriginalWOA(epoch=20, pop_size=15)
optimizer.solve(problem_dict, termination=term_dict)

# Cài đặt các tham số sau khi đã tối ưu
optimal_model = XGBRegressor(
    n_estimators=int(optimizer.g_best.solution[0]),
    learning_rate=optimizer.g_best.solution[1],
    max_depth=int(optimizer.g_best.solution[2]),
    # min_child_weight=optimizer.g_best.solution[3],
    gamma=optimizer.g_best.solution[3],
    random_state=42
)
optimal_model.fit(X_train, y_train)

# Lưu mô hình
joblib.dump(optimal_model, 'optimal_model_xgb_zoa')

# Đánh giá mô hình
y_pred_train = optimal_model.predict(X_train)
y_pred_test = optimal_model.predict(X_test)

rmse_train = mean_squared_error(y_train, y_pred_train) ** 0.5
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

rmse_test = mean_squared_error(y_test, y_pred_test) ** 0.5
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
print("Train:")
print("RMSE:", rmse_train, "MAE:", mae_train, "R2:", r2_train)

print("Test:")
print("RMSE:", rmse_test, "MAE:", mae_test, "R2:", r2_test)

# Lưu kết quả tối ưu
best_solution = np.array(optimizer.g_best.solution)
np.savetxt('best_solution_xgb_zoa.csv', best_solution, delimiter=',', comments='')

# Lưu kết quả dự đoán thành file CSV

# Tạo DataFrame cho train
df_train = pd.DataFrame(scaler.inverse_transform(X_train), columns=input_data.columns)
df_train['y_train'] = y_train.values
df_train['y_pred_train'] = y_pred_train
df_train.to_csv('train_results_xgb_zoa.csv', index=False)

# Tạo DataFrame cho test
df_test = pd.DataFrame(scaler.inverse_transform(X_test), columns=input_data.columns)
df_test['y_test'] = y_test.values
df_test['y_pred_test'] = y_pred_test
df_test.to_csv('test_results_xgb_zoa.csv', index=False)

2025/09/03 05:48:59 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: OriginalWOA(epoch=20, pop_size=15)
2025/09/03 05:49:05 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 1, Current best: 0.69, Global best: 0.69, Runtime: 3.24564 seconds
2025/09/03 05:49:08 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 2, Current best: 0.69, Global best: 0.69, Runtime: 2.98706 seconds
2025/09/03 05:49:11 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 3, Current best: 0.7, Global best: 0.7, Runtime: 2.32968 seconds
2025/09/03 05:49:13 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 4, Current best: 0.7, Global best: 0.7, Runtime: 2.33861 seconds
2025/09/03 05:49:16 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 5, Current best: 0.7, Global best: 0.7, Runtime: 2.62031 seconds
2025/09/03 05:49:20 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 6, Current best: 0.7, Global best: 0.7, Ru

Train:
RMSE: 1.226750781434255 MAE: 0.88359789629595 R2: 0.9852453556106118
Test:
RMSE: 3.5395088778243196 MAE: 2.817523917173728 R2: 0.9254793159269898


In [3]:
from xgboost import XGBRegressor
from mealpy import FloatVar, WOA
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Đọc dữ liệu
data = pd.read_csv('Data1.csv')

# Chia thành input và output
input_data = data.iloc[:, :-1]
output_data = data.iloc[:, -1]

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.15, random_state=37)

# Normalize
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, 'scaler')

# Hàm mục tiêu
def objective_func(x):
    n_estimators = int(x[0])
    learning_rate = x[1]
    max_depth = int(x[2])
    gamma = x[3]

    model = XGBRegressor(
        n_estimators=n_estimators, learning_rate=learning_rate,
        max_depth=max_depth, gamma=gamma, random_state=42
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
    return round(cv_scores.mean(), 4)

# Giới hạn tham số
lb = [50, 0.01, 1, 0]  
ub = [200, 0.3, 10, 1]

problem_dict = {
    "obj_func": objective_func,
    "bounds": FloatVar(lb=lb, ub=ub),
    "minmax": "max",
    "save_population": True
}

term_dict = {"max_early_stop": 10}

# Chạy huấn luyện 10 lần
results = []

for run in range(1, 11):
    print(f"\n====== LẦN HUẤN LUYỆN {run} ======")
    
    optimizer = WOA.OriginalWOA(epoch=20, pop_size=15)
    optimizer.solve(problem_dict, termination=term_dict)

    # Huấn luyện với best solution
    optimal_model = XGBRegressor(
        n_estimators=int(optimizer.g_best.solution[0]),
        learning_rate=optimizer.g_best.solution[1],
        max_depth=int(optimizer.g_best.solution[2]),
        gamma=optimizer.g_best.solution[3],
        random_state=42
    )
    optimal_model.fit(X_train, y_train)

    # Dự đoán
    y_pred_train = optimal_model.predict(X_train)
    y_pred_test = optimal_model.predict(X_test)

    # Đánh giá
    rmse_train = mean_squared_error(y_train, y_pred_train) ** 0.5
    mae_train = mean_absolute_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

    rmse_test = mean_squared_error(y_test, y_pred_test) ** 0.5
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    print("Train: RMSE =", rmse_train, " MAE =", mae_train, " R2 =", r2_train)
    print("Test:  RMSE =", rmse_test, " MAE =", mae_test, " R2 =", r2_test)

    results.append({
        "Run": run,
        "RMSE_train": rmse_train, "MAE_train": mae_train, "R2_train": r2_train,
        "RMSE_test": rmse_test, "MAE_test": mae_test, "R2_test": r2_test
    })

# Lưu toàn bộ kết quả 10 lần
df_results = pd.DataFrame(results)
df_results.to_csv("xgb_woa_10runs_results.csv", index=False)

print("\n===== TỔNG KẾT 10 LẦN =====")
print(df_results)


2025/09/03 05:50:41 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: OriginalWOA(epoch=20, pop_size=15)



====== LẦN HUẤN LUYỆN 1 ======


2025/09/03 05:50:47 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 1, Current best: 0.6856, Global best: 0.6856, Runtime: 2.67116 seconds
2025/09/03 05:50:49 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 2, Current best: 0.6856, Global best: 0.6856, Runtime: 2.09735 seconds
2025/09/03 05:50:51 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 3, Current best: 0.6928, Global best: 0.6928, Runtime: 2.17142 seconds
2025/09/03 05:50:53 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 4, Current best: 0.6979, Global best: 0.6979, Runtime: 2.19225 seconds
2025/09/03 05:50:57 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 5, Current best: 0.6982, Global best: 0.6982, Runtime: 3.84488 seconds
2025/09/03 05:51:01 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 6, Current best: 0.6982, Global best: 0.6982, Runtime: 4.61290 seconds
2025/09/03 05:51:05 PM, INFO, mealpy.swarm_bas

Train: RMSE = 0.8433209846424957  MAE = 0.6039171988810969  R2 = 0.9930272856405113
Test:  RMSE = 3.8796489963185015  MAE = 3.0085258826231347  R2 = 0.9104685348084566

====== LẦN HUẤN LUYỆN 2 ======


2025/09/03 05:51:42 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 1, Current best: 0.6969, Global best: 0.6969, Runtime: 2.20079 seconds
2025/09/03 05:51:45 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 2, Current best: 0.6976, Global best: 0.6976, Runtime: 2.77647 seconds
2025/09/03 05:51:48 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 3, Current best: 0.7003, Global best: 0.7003, Runtime: 2.46305 seconds
2025/09/03 05:51:50 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 4, Current best: 0.7022, Global best: 0.7022, Runtime: 1.88589 seconds
2025/09/03 05:51:52 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 5, Current best: 0.7022, Global best: 0.7022, Runtime: 2.04864 seconds
2025/09/03 05:51:54 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 6, Current best: 0.7033, Global best: 0.7033, Runtime: 2.22635 seconds
2025/09/03 05:51:56 PM, INFO, mealpy.swarm_bas

...

Train: RMSE = 1.262762970107812  MAE = 0.908359964353229  R2 = 0.9843663733576635
Test:  RMSE = 3.5401794535387543  MAE = 2.7581292049701394  R2 = 0.9254510767015076

====== LẦN HUẤN LUYỆN 9 ======


2025/09/03 05:57:08 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 1, Current best: 0.682, Global best: 0.682, Runtime: 2.11533 seconds
2025/09/03 05:57:10 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 2, Current best: 0.682, Global best: 0.682, Runtime: 2.23574 seconds
2025/09/03 05:57:13 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 3, Current best: 0.682, Global best: 0.682, Runtime: 2.25453 seconds
2025/09/03 05:57:15 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 4, Current best: 0.682, Global best: 0.682, Runtime: 2.19640 seconds
2025/09/03 05:57:17 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 5, Current best: 0.682, Global best: 0.682, Runtime: 2.35806 seconds
2025/09/03 05:57:20 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 6, Current best: 0.6855, Global best: 0.6855, Runtime: 2.79791 seconds
2025/09/03 05:57:23 PM, INFO, mealpy.swarm_based.WOA.Ori

Train: RMSE = 1.8086409882915835  MAE = 1.250942830339484  R2 = 0.9679283873665712
Test:  RMSE = 3.2723291273294373  MAE = 2.5262876266088243  R2 = 0.9363050828413737

====== LẦN HUẤN LUYỆN 10 ======


2025/09/03 05:58:05 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 1, Current best: 0.6741, Global best: 0.6741, Runtime: 1.61153 seconds
2025/09/03 05:58:07 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 2, Current best: 0.6741, Global best: 0.6741, Runtime: 1.40799 seconds
2025/09/03 05:58:09 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 3, Current best: 0.697, Global best: 0.697, Runtime: 2.13245 seconds
2025/09/03 05:58:11 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 4, Current best: 0.697, Global best: 0.697, Runtime: 2.01922 seconds
2025/09/03 05:58:12 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 5, Current best: 0.697, Global best: 0.697, Runtime: 1.62065 seconds
2025/09/03 05:58:14 PM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 6, Current best: 0.697, Global best: 0.697, Runtime: 1.63085 seconds
2025/09/03 05:58:16 PM, INFO, mealpy.swarm_based.WOA.O

Train: RMSE = 1.5389539567746737  MAE = 1.0830277819152272  R2 = 0.9767797274324743
Test:  RMSE = 3.5102099661209043  MAE = 2.87900575980162  R2 = 0.9267079260993992

===== TỔNG KẾT 10 LẦN =====
   Run  RMSE_train  MAE_train  R2_train  RMSE_test  MAE_test   R2_test
0    1    0.843321   0.603917  0.993027   3.879649  3.008526  0.910469
1    2    1.069574   0.743572  0.988784   3.666339  3.000780  0.920043
2    3    1.062571   0.753143  0.988930   3.664362  2.804480  0.920129
3    4    1.270753   0.889386  0.984168   3.430442  2.563690  0.930001
4    5    1.244452   0.855199  0.984816   4.076096  2.946140  0.901172
5    6    1.094526   0.763798  0.988255   4.083084  3.105034  0.900833
6    7    0.562040   0.399321  0.996903   3.417890  2.598216  0.930512
7    8    1.262763   0.908360  0.984366   3.540179  2.758129  0.925451
8    9    1.808641   1.250943  0.967928   3.272329  2.526288  0.936305
9   10    1.538954   1.083028  0.976780   3.510210  2.879006  0.926708
